<a href="https://colab.research.google.com/github/HSS107048212/ELEBC_Youbike/blob/main/05_AC_PV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get data by API

In [6]:
import requests

def fetch_youbike_data(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print("Failed to fetch data. Status code:", response.status_code)
            return None
    except Exception as e:
        print("An error occurred:", e)
        return None

if __name__ == "__main__":
    url = "https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json"
    youbike_data = fetch_youbike_data(url)
    if youbike_data:
        print("YouBike data retrieved successfully!")
        # 在这里对您的YouBike数据进行处理
    else:
        print("Failed to fetch YouBike data.")

import pandas as pd

# Convert JSON data to DataFrame
df = pd.DataFrame(youbike_data)

from datetime import datetime

current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d")
print("Current system time (YYYY-MM-DD):", formatted_time)
selected_columns = ['sno', 'sna', 'tot','infoDate']
# 创建新的DataFrame，只包含选定的列
df= df[selected_columns]

YouBike data retrieved successfully!
Current system time (YYYY-MM-DD): 2024-04-23


## Compute AC

In [13]:
import numpy as np

Rides_day_for_breakeven = 93667
# Why Rides_day_for_breakeven is 93667? you can check on https://docs.google.com/presentation/d/1tBNcwg75QwaClUM5m2wP-YxiqWZI_SR3Ztjk6YZLKsw/edit#slide=id.g26e84c87d13_0_209
df["AC"] = np.ceil(Rides_day_for_breakeven * df["tot"] / sum(df["tot"]))
df

,sno,sna,tot,infoDate,AC
0,500101001,YouBike2.0_捷運科技大樓站,28,2024-04-23,76.0
1,500101002,YouBike2.0_復興南路二段273號前,21,2024-04-23,57.0
2,500101003,YouBike2.0_國北教大實小東側門,16,2024-04-23,44.0
3,500101004,YouBike2.0_和平公園東側,11,2024-04-23,30.0
4,500101005,YouBike2.0_辛亥復興路口西北側,16,2024-04-23,44.0
...,...,...,...,...,...
1406,500119087,YouBike2.0_臺大總圖書館西南側,30,2024-04-23,82.0
1407,500119088,YouBike2.0_臺大黑森林西側,20,2024-04-23,55.0
1408,500119089,YouBike2.0_臺大獸醫館南側,24,2024-04-23,65.0
1409,500119090,YouBike2.0_臺大新體育館東南側,40,2024-04-23,109.0


## Compute PV

In [15]:
Expected_usage_percentage = 0.05
# Why Expected_usage_percentage is 0.05 ? you can check on https://docs.google.com/presentation/d/1tBNcwg75QwaClUM5m2wP-YxiqWZI_SR3Ztjk6YZLKsw/edit#slide=id.g26e84c87d13_0_242
df["PV"] = np.ceil(df["tot"] * Expected_usage_percentage *95)
df

,sno,sna,tot,infoDate,AC,PV
0,500101001,YouBike2.0_捷運科技大樓站,28,2024-04-23,76.0,133.0
1,500101002,YouBike2.0_復興南路二段273號前,21,2024-04-23,57.0,100.0
2,500101003,YouBike2.0_國北教大實小東側門,16,2024-04-23,44.0,76.0
3,500101004,YouBike2.0_和平公園東側,11,2024-04-23,30.0,53.0
4,500101005,YouBike2.0_辛亥復興路口西北側,16,2024-04-23,44.0,76.0
...,...,...,...,...,...,...
1406,500119087,YouBike2.0_臺大總圖書館西南側,30,2024-04-23,82.0,143.0
1407,500119088,YouBike2.0_臺大黑森林西側,20,2024-04-23,55.0,95.0
1408,500119089,YouBike2.0_臺大獸醫館南側,24,2024-04-23,65.0,115.0
1409,500119090,YouBike2.0_臺大新體育館東南側,40,2024-04-23,109.0,190.0


## Import AC_PV file to Google drive

In [17]:
df.to_excel(f"{formatted_time}_Youbike_AC_PV.xlsx", index=False)

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

directory_path = '/content/'
# C:/Users/Elebc_GPU_Server/Desktop/排程抓資料/
# /content/

# 設置Google Drive API的認證
credentials = service_account.Credentials.from_service_account_file(
    f'{directory_path}rich-operand-418201-63c868c738c8.json',  # 請替換成你的認證文件路徑
    scopes=['https://www.googleapis.com/auth/drive']
)

# 創建Google Drive API的服務
drive_service = build('drive', 'v3', credentials=credentials)


# 要上傳的Excel文件路徑
excel_file_path = f'{directory_path}{formatted_time}_Youbike_AC_PV.xlsx'  # 請替換成你的Excel文件路徑

# 要上傳到的Google Drive文件夾的ID
folder_id = '1aeTv-6l0nfmpEAfKHBvd7l5p0l9Kfplb'  # 請替換成目標文件夾的ID

import os

# 上傳文件
file_metadata = {
    'name': os.path.basename(excel_file_path),
    'parents': [folder_id]
}
media = MediaFileUpload(excel_file_path, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

print('File ID: %s' % file.get('id'))

File ID: 1I1Kks959G3Jhnb9vM1nKZSpDTwyxvBvy
